
# 🌼 대규모 LLM을 활용한 지식 챗봇 개발 - 3차시(24.11.25)

---

In [ ]:
import tensorflow as tf
from transformers import AutoTokenizer, TFGPT2LMHeadModel
import pandas as pd
import tqdm

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', pad_token='<pad>')
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.0.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'lm_head.weight', 'transformer.h.3.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.4.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_data = pd.read_excel('/content/drive/MyDrive/ChatbotData.xlsx')

In [ ]:
print(f'데이터 개수: {len(train_data)}')

데이터 개수: 11823


In [ ]:
train_data.head()

,Q,A
0,12시 땡!,하루가 또 가네요.
1,1지망 학교 떨어졌어,위로해 드립니다.
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.
4,PPL 심하네,눈살이 찌푸려지죠.


In [ ]:
def get_chat_data():
  for question, answer in zip(train_data.Q.to_list(), train_data.A.to_list()):
    bos_token = [tokenizer.bos_token_id]  # bos_token: 문장이 시작될 때의 토큰
    eos_token = [tokenizer.eos_token_id]  # eos_token: 문장이 끝날 때의 토큰
    sent = tokenizer.encode('<usr>' + question + '<sys>' + answer)
    yield bos_token + sent + eos_token  # yield: 데이터가 필요할 때마다 return (메모리 효율 good)

In [ ]:
batch_size = 16
dataset = tf.data.Dataset.from_generator(get_chat_data, output_signature=tf.TensorSpec(shape=(None,), dtype=tf.int32))  # TensorSpec: tensor로 변환할 때 입력값 None(지정X)

In [ ]:
dataset = dataset.padded_batch(batch_size=batch_size, padded_shapes=(None, ), padding_values=tokenizer.pad_token_id)

In [ ]:
for batch in dataset:
  print(batch)
  break

# 시작 토큰과 끝 토큰을 같게 설정했지만 인덱스는 다르다? 1인데...?

tf.Tensor(
[[    1     2  9349  7888   739  7318   376     4 12557  6824  9108  9028
   7098 25856     1     3     3     3     3     3     3]
 [    1     2  9020  8263  7497 10192 11615  8210  8006     4 12422  8711
   9535  7483 12521     1     3     3     3     3     3]
 [    1     2  9085  7597   395  8149 10624  7397 24224 13358  7182     4
  12079  8135 16899  9677  8234   389     1     3     3]
 [    1     2  9085  7597   395  8149  9465 10624  7397 24224 13358  7182
      4 12079  8135 16899  9677  8234   389     1     3]
 [    1     2  9943   422   418  9327  8702  7098     4  9847 16912 18328
   8671  7415  8263  8234   389     1     3     3     3]
 [    1     2  9815   410 21249 10174  6824  8210  8006     4  9427 11056
  11594 10137 10556  9266  8711 25856     1     3     3]
 [    1     2  9815   410 21249  9183  7249     4  9427 11056 11594 10137
  10556  9266  8711 25856     1     3     3     3     3]
 [    1     2  9815 37655  9622  8619 10401  9183  9328   216     4  944

In [ ]:
print(tokenizer.decode(batch[0]))
[1, 2, 9349, 7888, 739, 7318, 376, 4, 12557, 6824, 9108, 9028, 7098, 25856, 1, 3, 3, 3, 3, 3, 3]

</s><usr> 12시 땡!<sys> 하루가 또 가네요.</s><pad><pad><pad><pad><pad><pad>


[1,
 2,
 9349,
 7888,
 739,
 7318,
 376,
 4,
 12557,
 6824,
 9108,
 9028,
 7098,
 25856,
 1,
 3,
 3,
 3,
 3,
 3,
 3]

In [ ]:
adam = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)
# epsilon: 0으로 나눠지는 상황을 방지하기 위해 아주 작은 값을 추가 (모델의 안정성 증가)

In [ ]:
steps = len(train_data) // batch_size + 1  # 남는 데이터가 있을 경우를 대비해 스텝을 추가하여 학습 (배치 사이즈로 딱 나누어 떨어지지 않을 수 있다)

In [ ]:
epochs = 3
for epochs in range(epochs):
    epoch_loss = 0
    for batch in tqdm.tqdm(dataset, total=steps):  # tqdm으로 시각화
        with tf.GradientTape() as tape:
            result = model(batch, labels=batch)  # 교사강요, labels=batch: 정답 데이터, 시퀀스가 하나 shift된 상태
            loss = result[0]
            batch_loss = tf.reduce_mean(loss)  # loss 값을 평균화 해서 batch_loss에 입력
        grads = tape.gradient(batch_loss, model.trainable_variables)
        # 학습 가능한 변수 trainable_variables의 기울기를 계산
        adam.apply_gradients(zip(grads, model.trainable_variables))
        epoch_loss += batch_loss/steps  # epoch의 loss 값
    print(f'[Epoch : {epochs+1}] cost = {epoch_loss:>.9}')

100%|██████████| 739/739 [09:21<00:00,  1.32it/s]


[Epoch : 1] cost = 2.14646912


100%|██████████| 739/739 [09:21<00:00,  1.32it/s]


[Epoch : 2] cost = 1.71464109


100%|██████████| 739/739 [10:21<00:00,  1.19it/s]

[Epoch : 3] cost = 1.33611929


In [ ]:
text = '오늘 뭐하지'
sent = '<usr>' + text + '<sys>'
input_ids = [tokenizer.bos_token_id] + tokenizer.encode(sent)
input_ids = tf.convert_to_tensor([input_ids])

In [ ]:
print(f'정수 인코딩: {input_ids}')

정수 인코딩: [[    1     2 10070 46651  9328     4]]


In [ ]:
print(f'정수 인코딩을 디코딩: {tokenizer.decode(input_ids[0])}')

정수 인코딩을 디코딩: </s><usr> 오늘 뭐하지<sys>


In [ ]:
output = model.generate(input_ids, max_length=50, early_stopping=True, eos_token_id=tokenizer.eos_token_id)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:638: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


In [ ]:
decoded_sentence = tokenizer.decode(output[0].numpy().tolist())
print(decoded_sentence)

</s><usr> 오늘 뭐하지<sys> 오늘은 그냥 연락하고 마음으로 키우는 거죠.</s>


In [ ]:
print(decoded_sentence.split('<sys> ')[1].replace('</s>', ''))  # 답변만 출력하기 위해 1번째 인덱스

오늘은 그냥 연락하고 마음으로 키우는 거죠.


In [ ]:
output = model.generate(input_ids, max_length=100, do_sample=True, top_k=10)
# do_sample=True: 텍스트 생성 시 확률 기바느올 랜덤성을 추가
decoded_sentence = tokenizer.decode(output[0].numpy().tolist())
print(decoded_sentence.split('<sys> ')[1].replace('</s>', ''))

나랑 같이 놀아요.


In [ ]:
def return_answer(user_text):
    sent = '<usr>' + user_text + '<sys>'
    input_ids = [tokenizer.bos_token_id] + tokenizer.encode(sent)
    input_ids = tf.convert_to_tensor([input_ids])
    output = model.generate(input_ids, max_length=50, do_sample=True, top_k=3)
    sentence = tokenizer.decode(output[0].numpy().tolist())
    chatbot_respone = sentence.split('<sys> ')[1].replace('</s>', '')
    return chatbot_respone

In [ ]:
return_answer('오늘 뭐먹지?')

'오늘 먹는 거 좋아해서 못 먹는다고 하셨는데 맛난 거 같아요.'

In [ ]:
return_answer('책 추천해줘')

'책 소개가 좋겠어요.'

In [ ]:
return_answer('나 누구게')

'그 사람의 마음을 알기는 어려울 거 같아요.'